In [16]:
import os
import tarfile
import gzip
import shutil
import re
from pathlib import Path

import scanpy as sc


In [11]:
# -------------------------------
# Step 0: Define paths (absolute, robust)
# -------------------------------
base_dir = Path.cwd()  # /home/tsaliu (from your output)

tar_file = base_dir / "GSE159977_RAW.tar"
extract_folder = base_dir / "GSE159977_RAW"
output_folder = base_dir / "GSE159977_h5ad"

output_folder.mkdir(parents=True, exist_ok=True)
extract_folder.mkdir(parents=True, exist_ok=True)

print("Base dir       :", base_dir)
print("Tar file       :", tar_file)
print("Extract folder :", extract_folder)
print("Output folder  :", output_folder)

if not tar_file.exists():
    raise FileNotFoundError(f"Cannot find tar file: {tar_file}")

Base dir       : /home/tsaliu
Tar file       : /home/tsaliu/GSE159977_RAW.tar
Extract folder : /home/tsaliu/GSE159977_RAW
Output folder  : /home/tsaliu/GSE159977_h5ad


In [12]:
# -------------------------------
# Step 1: Extract tar (skip if already extracted)
# -------------------------------
def is_extract_nonempty(folder: Path) -> bool:
    try:
        return any(folder.iterdir())
    except FileNotFoundError:
        return False

if not is_extract_nonempty(extract_folder):
    print("\n[1/5] Extracting tar file...")
    with tarfile.open(tar_file, "r:*") as tar:
        tar.extractall(path=extract_folder)
    print("Extraction done.")
else:
    print("\n[1/5] Extraction folder already has content; skipping tar extraction.")


[1/5] Extracting tar file...
Extraction done.


In [13]:
# -------------------------------
# Step 2: Unzip .gz files recursively (idempotent)
# -------------------------------
print("\n[2/5] Unzipping .gz files (only if needed)...")

gz_files = list(extract_folder.rglob("*.gz"))
print(f"Found {len(gz_files)} .gz files")

unzipped_count = 0
for gz_path in gz_files:
    out_path = gz_path.with_suffix("")  # remove .gz
    if out_path.exists():
        continue  # already unzipped
    with gzip.open(gz_path, "rb") as f_in, open(out_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    unzipped_count += 1

print(f"Unzipped {unzipped_count} files (skipped {len(gz_files) - unzipped_count} already present).")



[2/5] Unzipping .gz files (only if needed)...
Found 30 .gz files
Unzipped 30 files (skipped 0 already present).


In [23]:
# -------------------------------
# Step 3: Map GSM to PT labels
# -------------------------------
gsm_to_pt = {
    "GSM4851987": "PT-5",
    "GSM4851988": "PT-8",
    "GSM4851989": "PT-9",
    "GSM4851990": "PT-11",
    "GSM4851991": "PT-12",
    "GSM4851992": "PT-16",
    "GSM4851993": "PT-17",
    "GSM4851994": "PT-19",
    "GSM4851995": "PT-20",
    "GSM4851996": "PT-21",
}

In [24]:
# -------------------------------
# Step 4: Find valid 10x directories
# -------------------------------
print("\n[3/5] Locating valid 10x directories...")

def is_10x_dir(d: Path) -> bool:
    """A directory is considered a 10x mtx directory if it has
    - matrix.mtx (or matrix.mtx.gz, but we unzipped)
    - barcodes.tsv (or barcodes.tsv.gz)
    - features.tsv OR genes.tsv (new vs old 10x)
    """
    if not d.is_dir():
        return False
    has_matrix = (d / "matrix.mtx").exists()
    has_barcodes = (d / "barcodes.tsv").exists()
    has_features = (d / "features.tsv").exists() or (d / "genes.tsv").exists()
    return has_matrix and has_barcodes and has_features

# search all subdirectories
candidate_dirs = [p for p in extract_folder.rglob("*") if p.is_dir()]
tenx_dirs = [d for d in candidate_dirs if is_10x_dir(d)]

# If nothing found, sometimes the files are in the extract_folder itself
if is_10x_dir(extract_folder):
    tenx_dirs.insert(0, extract_folder)

# De-duplicate (in case extract_folder already included)
tenx_dirs = sorted(set(tenx_dirs))

print(f"Found {len(tenx_dirs)} valid 10x directories.")
for d in tenx_dirs[:10]:
    print("  -", d)
if len(tenx_dirs) > 10:
    print("  ...")

if len(tenx_dirs) == 0:
    raise RuntimeError(
        "No valid 10x directories found. "
        "Check what was extracted/unzipped (e.g., list extract_folder contents)."
    )



[3/5] Locating valid 10x directories...
Found 10 valid 10x directories.
  - /home/tsaliu/GSE159977_RAW/GSM4851987
  - /home/tsaliu/GSE159977_RAW/GSM4851988
  - /home/tsaliu/GSE159977_RAW/GSM4851989
  - /home/tsaliu/GSE159977_RAW/GSM4851990
  - /home/tsaliu/GSE159977_RAW/GSM4851991
  - /home/tsaliu/GSE159977_RAW/GSM4851992
  - /home/tsaliu/GSE159977_RAW/GSM4851993
  - /home/tsaliu/GSE159977_RAW/GSM4851994
  - /home/tsaliu/GSE159977_RAW/GSM4851995
  - /home/tsaliu/GSE159977_RAW/GSM4851996


In [25]:
# -------------------------------
# Step 5: Load each sample + save per-sample h5ad
# -------------------------------
print("\n[4/5] Loading samples and writing per-sample .h5ad files...")

adata_list = []

def infer_sample_id_from_path(p: Path) -> str:
    # Try to find a GSM ID in the path components
    for part in p.parts[::-1]:
        if part.startswith("GSM"):
            return part
    # fallback: directory name
    return p.name

for folder in tenx_dirs:
    sample_id = infer_sample_id_from_path(folder)  # e.g. GSM4851987
    pt_label = gsm_to_pt.get(sample_id, sample_id)

    print(f"\nLoading: {folder}")
    print(f"Sample : {sample_id} -> {pt_label}")

    try:
        ad = sc.read_10x_mtx(
            folder,
            var_names="gene_symbols",
            cache=False  # safer on shared FS; enable if you want
        )
    except Exception as e:
        print(f"  !! Failed to read 10x from {folder}: {e}")
        continue

    # Add metadata
    ad.obs["sample"] = pt_label
    ad.obs["gsm"] = sample_id
    ad.obs["dataset"] = "GSE159977"

    # Optional: make obs/var names unique (often helpful)
    ad.obs_names_make_unique()
    ad.var_names_make_unique()

    # Save per-sample
    per_sample_file = output_folder / f"GSE159977_{pt_label}.h5ad"
    ad.write(per_sample_file)
    print(f"Saved: {per_sample_file}")

    adata_list.append(ad)

print(f"\nLoaded and saved {len(adata_list)} samples.")


[4/5] Loading samples and writing per-sample .h5ad files...

Loading: /home/tsaliu/GSE159977_RAW/GSM4851987
Sample : GSM4851987 -> PT-5
  !! Failed to read 10x from /home/tsaliu/GSE159977_RAW/GSM4851987: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851987/matrix.mtx.gz.

Loading: /home/tsaliu/GSE159977_RAW/GSM4851988
Sample : GSM4851988 -> PT-8
  !! Failed to read 10x from /home/tsaliu/GSE159977_RAW/GSM4851988: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851988/matrix.mtx.gz.

Loading: /home/tsaliu/GSE159977_RAW/GSM4851989
Sample : GSM4851989 -> PT-9
  !! Failed to read 10x from /home/tsaliu/GSE159977_RAW/GSM4851989: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851989/matrix.mtx.gz.

Loading: /home/tsaliu/GSE159977_RAW/GSM4851990
Sample : GSM4851990 -> PT-11
  !! Failed to read 10x from /home/tsaliu/GSE159977_RAW/GSM4851990: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851990/matrix.mtx.gz.

Loading: /home/tsaliu/GSE159977_RAW/GSM4851991
Sample : GSM4851991 -> PT-12
 

In [26]:
# -------------------------------
# Step 6 (optional): concatenate and save combined dataset
# -------------------------------
print("\n[5/5] (Optional) Concatenating into one AnnData...")

if len(adata_list) > 0:
    adata_all = adata_list[0].concatenate(
        adata_list[1:],
        batch_key="batch",
        batch_categories=[a.obs["sample"].unique()[0] for a in adata_list],
        index_unique=None
    )
    combined_file = output_folder / "GSE159977_all_samples.h5ad"
    adata_all.write(combined_file)
    print(f"Saved combined file: {combined_file}")
else:
    print("No samples loaded successfully; skipping concatenation.")


[5/5] (Optional) Concatenating into one AnnData...
No samples loaded successfully; skipping concatenation.


In [28]:
import os
import re
import shutil
from pathlib import Path
import scanpy as sc

extract_folder = Path("GSE159977_RAW")
output_folder = Path("GSE159977_h5ad")
output_folder.mkdir(exist_ok=True)

gsm_to_pt = {
    "GSM4851987": "PT-5",
    "GSM4851988": "PT-8",
    "GSM4851989": "PT-9",
    "GSM4851990": "PT-11",
    "GSM4851991": "PT-12",
    "GSM4851992": "PT-16",
    "GSM4851993": "PT-17",
    "GSM4851994": "PT-19",
    "GSM4851995": "PT-20",
    "GSM4851996": "PT-21",
}

gsm_folders = sorted([p for p in extract_folder.iterdir() if p.is_dir() and p.name.startswith("GSM")])

def pick(files, pattern):
    for f in files:
        if re.search(pattern, f.name, flags=re.IGNORECASE):
            return f
    return None

def link_or_copy(dst: Path, src: Path):
    """
    Make dst point to src.
    Prefer symlink; if that fails (HPC FS), copy.
    """
    if dst.exists() or dst.is_symlink():
        return
    try:
        dst.symlink_to(src)
    except Exception:
        shutil.copy2(src, dst)

print(f"Found {len(gsm_folders)} GSM folders. Forcing expected filenames...")

for folder in gsm_folders:
    files = [p for p in folder.iterdir() if p.is_file()]

    # find the 3 components by pattern (handles GSM-prefixed names)
    mtx = pick(files, r"\.mtx(\.gz)?$")
    bar = pick(files, r"barcodes.*\.tsv(\.gz)?$")
    feat = pick(files, r"(features|genes).*\.tsv(\.gz)?$")

    if not (mtx and bar and feat):
        print("Missing trio in", folder.name, "files:", [f.name for f in files])
        continue

    # Force both gz + non-gz standard names to exist (whichever scanpy wants)
    link_or_copy(folder / "matrix.mtx", mtx)
    link_or_copy(folder / "barcodes.tsv", bar)
    link_or_copy(folder / "features.tsv", feat)

    link_or_copy(folder / "matrix.mtx.gz", mtx)
    link_or_copy(folder / "barcodes.tsv.gz", bar)
    link_or_copy(folder / "features.tsv.gz", feat)

print("Done forcing names.\n")

# Quick verification: show what's in one folder now
test_folder = gsm_folders[0]
print("Example folder:", test_folder)
print(sorted([p.name for p in test_folder.iterdir() if p.is_file()])[:30])

# Now load + save
loaded = 0
for folder in gsm_folders:
    gsm = folder.name
    pt = gsm_to_pt.get(gsm, gsm)

    try:
        ad = sc.read_10x_mtx(folder, var_names="gene_symbols", cache=False)
    except Exception as e:
        print(f"FAILED {gsm}: {e}")
        continue

    ad.obs["sample"] = pt
    ad.obs["gsm"] = gsm
    ad.obs["dataset"] = "GSE159977"
    ad.obs_names_make_unique()
    ad.var_names_make_unique()

    out = output_folder / f"GSE159977_{pt}.h5ad"
    ad.write(out)
    print("Saved:", out)
    loaded += 1

print("\nLoaded", loaded, "samples.")


Found 10 GSM folders. Forcing expected filenames...
Done forcing names.

Example folder: GSE159977_RAW/GSM4851987
['barcodes.tsv', 'features.tsv', 'matrix.mtx']
FAILED GSM4851987: Did not find file GSE159977_RAW/GSM4851987/matrix.mtx.gz.
FAILED GSM4851988: Did not find file GSE159977_RAW/GSM4851988/matrix.mtx.gz.
FAILED GSM4851989: Did not find file GSE159977_RAW/GSM4851989/matrix.mtx.gz.
FAILED GSM4851990: Did not find file GSE159977_RAW/GSM4851990/matrix.mtx.gz.
FAILED GSM4851991: Did not find file GSE159977_RAW/GSM4851991/matrix.mtx.gz.
FAILED GSM4851992: Did not find file GSE159977_RAW/GSM4851992/matrix.mtx.gz.
FAILED GSM4851993: Did not find file GSE159977_RAW/GSM4851993/matrix.mtx.gz.
FAILED GSM4851994: Did not find file GSE159977_RAW/GSM4851994/matrix.mtx.gz.
FAILED GSM4851995: Did not find file GSE159977_RAW/GSM4851995/matrix.mtx.gz.
FAILED GSM4851996: Did not find file GSE159977_RAW/GSM4851996/matrix.mtx.gz.

Loaded 0 samples.


In [30]:
import os
import gzip
import shutil
from pathlib import Path
import scanpy as sc

# Input extracted GSM folders
extract_folder = Path.cwd() / "GSE159977_RAW"

# Output prepared 10x dirs (new, writable)
prepared_root = Path.cwd() / "GSE159977_10x_prepared"
prepared_root.mkdir(parents=True, exist_ok=True)

# Output h5ad
output_folder = Path.cwd() / "GSE159977_h5ad"
output_folder.mkdir(parents=True, exist_ok=True)

gsm_to_pt = {
    "GSM4851987": "PT-5",
    "GSM4851988": "PT-8",
    "GSM4851989": "PT-9",
    "GSM4851990": "PT-11",
    "GSM4851991": "PT-12",
    "GSM4851992": "PT-16",
    "GSM4851993": "PT-17",
    "GSM4851994": "PT-19",
    "GSM4851995": "PT-20",
    "GSM4851996": "PT-21",
}

gsm_folders = sorted([p for p in extract_folder.iterdir() if p.is_dir() and p.name.startswith("GSM")])
print("Found GSM folders:", len(gsm_folders))

def gzip_copy(src: Path, dst_gz: Path):
    dst_gz.parent.mkdir(parents=True, exist_ok=True)
    with open(src, "rb") as f_in, gzip.open(dst_gz, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

prepared_dirs = []

# 1) Prepare gzipped 10x files into a new tree
for folder in gsm_folders:
    gsm = folder.name

    # confirm source files exist
    mtx = folder / "matrix.mtx"
    bar = folder / "barcodes.tsv"
    feat = folder / "features.tsv"

    if not (mtx.exists() and bar.exists() and feat.exists()):
        print(f"Skipping {gsm}: missing one of matrix.mtx/barcodes.tsv/features.tsv")
        print("  files:", sorted([p.name for p in folder.iterdir() if p.is_file()]))
        continue

    outdir = prepared_root / gsm
    outdir.mkdir(parents=True, exist_ok=True)

    # Create gz versions (overwrite if you want by deleting first; here we skip if exists)
    if not (outdir / "matrix.mtx.gz").exists():
        gzip_copy(mtx, outdir / "matrix.mtx.gz")
    if not (outdir / "barcodes.tsv.gz").exists():
        gzip_copy(bar, outdir / "barcodes.tsv.gz")
    if not (outdir / "features.tsv.gz").exists():
        gzip_copy(feat, outdir / "features.tsv.gz")

    prepared_dirs.append(outdir)

print("Prepared dirs:", len(prepared_dirs))
print("Example prepared contents:", sorted([p.name for p in prepared_dirs[0].iterdir()]))

# 2) Read and save h5ad
loaded = 0
adata_list = []

for folder in prepared_dirs:
    gsm = folder.name
    pt = gsm_to_pt.get(gsm, gsm)

    try:
        ad = sc.read_10x_mtx(folder, var_names="gene_symbols", cache=False)
    except Exception as e:
        print(f"FAILED {gsm}: {e}")
        continue

    ad.obs["sample"] = pt
    ad.obs["gsm"] = gsm
    ad.obs["dataset"] = "GSE159977"
    ad.obs_names_make_unique()
    ad.var_names_make_unique()

    out = output_folder / f"GSE159977_{pt}.h5ad"
    ad.write(out)
    print("Saved:", out)
    adata_list.append(ad)
    loaded += 1

print("\nLoaded", loaded, "samples.")


Found GSM folders: 10
Prepared dirs: 10
Example prepared contents: ['barcodes.tsv.gz', 'features.tsv.gz', 'matrix.mtx.gz']
FAILED GSM4851987: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851988: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851989: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851990: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851991: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851992: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851993: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851994: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851995: Line 1: Not a Matrix Market file. Missing banner.
FAILED GSM4851996: Line 1: Not a Matrix Market file. Missing banner.

Loaded 0 samples.


In [31]:
from pathlib import Path

p = Path("GSE159977_RAW/GSM4851987/matrix.mtx")
b = p.read_bytes()[:4]
print("First 4 bytes:", b)
print("Looks gzipped?" , b[:2] == b"\x1f\x8b")

# If it's plain text, print first line; if gzipped, don't print text
if b[:2] != b"\x1f\x8b":
    with open(p, "rt", errors="replace") as f:
        print("First line:", f.readline().strip())


First 4 bytes: b'\x1f\x8b\x08\x08'
Looks gzipped? True


In [32]:
import os
from pathlib import Path
import scanpy as sc

extract_folder = Path.cwd() / "GSE159977_RAW"
output_folder = Path.cwd() / "GSE159977_h5ad"
output_folder.mkdir(exist_ok=True)

gsm_to_pt = {
    "GSM4851987": "PT-5",
    "GSM4851988": "PT-8",
    "GSM4851989": "PT-9",
    "GSM4851990": "PT-11",
    "GSM4851991": "PT-12",
    "GSM4851992": "PT-16",
    "GSM4851993": "PT-17",
    "GSM4851994": "PT-19",
    "GSM4851995": "PT-20",
    "GSM4851996": "PT-21",
}

gsm_folders = sorted([p for p in extract_folder.iterdir() if p.is_dir() and p.name.startswith("GSM")])
print("Found GSM folders:", len(gsm_folders))

def safe_link_or_copy(dst: Path, src: Path):
    """
    Create dst pointing to src.
    Prefer symlink; if not possible, copy.
    """
    if dst.exists() or dst.is_symlink():
        return
    try:
        dst.symlink_to(src)
    except Exception:
        # symlink not allowed -> copy the file bytes
        dst.write_bytes(src.read_bytes())

# 1) Create expected .gz filenames pointing to the existing gzipped-but-misnamed files
for folder in gsm_folders:
    mtx  = folder / "matrix.mtx"      # actually gzipped already
    bar  = folder / "barcodes.tsv"    # likely gzipped too (GEO often does this)
    feat = folder / "features.tsv"    # likely gzipped too

    if not (mtx.exists() and bar.exists() and feat.exists()):
        print("Skipping", folder.name, "files:", [p.name for p in folder.iterdir() if p.is_file()])
        continue

    safe_link_or_copy(folder / "matrix.mtx.gz", mtx)
    safe_link_or_copy(folder / "barcodes.tsv.gz", bar)
    safe_link_or_copy(folder / "features.tsv.gz", feat)

print("Created expected .gz filenames in each GSM folder.")

# 2) Load + write per-sample h5ad
loaded = 0
adata_list = []

for folder in gsm_folders:
    gsm = folder.name
    pt = gsm_to_pt.get(gsm, gsm)

    try:
        ad = sc.read_10x_mtx(folder, var_names="gene_symbols", cache=False)
    except Exception as e:
        print(f"FAILED {gsm}: {e}")
        continue

    ad.obs["sample"] = pt
    ad.obs["gsm"] = gsm
    ad.obs["dataset"] = "GSE159977"
    ad.obs_names_make_unique()
    ad.var_names_make_unique()

    out = output_folder / f"GSE159977_{pt}.h5ad"
    ad.write(out)
    print("Saved:", out)

    adata_list.append(ad)
    loaded += 1

print("\nLoaded", loaded, "samples.")


Found GSM folders: 10
Created expected .gz filenames in each GSM folder.
FAILED GSM4851987: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851987/matrix.mtx.gz.
FAILED GSM4851988: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851988/matrix.mtx.gz.
FAILED GSM4851989: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851989/matrix.mtx.gz.
FAILED GSM4851990: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851990/matrix.mtx.gz.
FAILED GSM4851991: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851991/matrix.mtx.gz.
FAILED GSM4851992: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851992/matrix.mtx.gz.
FAILED GSM4851993: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851993/matrix.mtx.gz.
FAILED GSM4851994: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851994/matrix.mtx.gz.
FAILED GSM4851995: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851995/matrix.mtx.gz.
FAILED GSM4851996: Did not find file /home/tsaliu/GSE159977_RAW/GSM4851996/matrix.mtx.gz.

Loaded 0 samples.
